<a href="https://colab.research.google.com/github/alfazick/AppliedLLMCourse/blob/main/ProjectMultiModalInferenceOnly2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128,expandable_segments:True"
os.environ["PYTORCH_NO_CUDA_MEMORY_CACHING"] = "1"

In [ ]:
# Cell 2 - MUST restart kernel
import os
os._exit(0)  # This forces kernel restart

In [ ]:
import torch
print("alloc conf:", os.environ.get("PYTORCH_CUDA_ALLOC_CONF"))


alloc conf: max_split_size_mb:128,expandable_segments:True


In [ ]:
import importlib

# Verify installed versions match requirements
requirements = {
    'torch': '2.4.1',
    'transformers': '4.46.2',
    'trl': '0.11.4',
    'datasets': '3.0.2',
    'bitsandbytes': '0.44.1',
    'peft': '0.13.2',
    'qwen_vl_utils': '0.0.8',
    'wandb': '0.18.5',
    'accelerate': '1.0.1'
}

print("=== VERSION CHECK ===")
for package, expected in requirements.items():
    try:
        module = importlib.import_module(package)
        actual = getattr(module, '__version__', 'unknown')
        status = "✅" if actual == expected else "⚠️"
        print(f"{status} {package}: {actual} (expected: {expected})")
    except ImportError:
        print(f"❌ {package}: NOT INSTALLED (expected: {expected})")

=== VERSION CHECK ===
⚠️ torch: 2.8.0+cu128 (expected: 2.4.1)
✅ transformers: 4.46.2 (expected: 4.46.2)
✅ trl: 0.11.4 (expected: 0.11.4)
✅ datasets: 3.0.2 (expected: 3.0.2)
⚠️ bitsandbytes: 0.48.2 (expected: 0.44.1)
✅ peft: 0.13.2 (expected: 0.13.2)
⚠️ qwen_vl_utils: unknown (expected: 0.0.8)
✅ wandb: 0.18.5 (expected: 0.18.5)
✅ accelerate: 1.0.1 (expected: 1.0.1)


In [ ]:
import gc
import time

def clear_memory():
    # Delete variables if they exist in the current global scope
    if 'inputs' in globals(): del globals()['inputs']
    if 'model' in globals(): del globals()['model']
    if 'processor' in globals(): del globals()['processor']
    if 'trainer' in globals(): del globals()['trainer']
    if 'peft_model' in globals(): del globals()['peft_model']
    if 'bnb_config' in globals(): del globals()['bnb_config']
    time.sleep(2)

    # Garbage collection and clearing CUDA memory
    gc.collect()
    time.sleep(2)
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    time.sleep(2)
    gc.collect()
    time.sleep(2)

    print(f"GPU allocated memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU reserved memory: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

clear_memory()

GPU allocated memory: 0.00 GB
GPU reserved memory: 0.00 GB


In [ ]:
!nvidia-smi

Sat Nov 22 00:40:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.06             Driver Version: 570.124.06     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5090        On  |   00000000:01:00.0 Off |                  N/A |
|  0%   33C    P1             66W /  575W |     506MiB /  32607MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import torch
import os
from matplotlib import pyplot as plt
from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor,AutoProcessor
from qwen_vl_utils import vision_process,process_vision_info
import peft
import bitsandbytes as bnb
from transformers import BitsAndBytesConfig


In [ ]:
# NF4 quantization config
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

quantization_config=BitsAndBytesConfig(
        load_in_8bit=True,
        llm_int8_threshold=6.0
    )

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")



model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    quantization_config=quantization_config,
    device_map="auto"
)
print(model.device)
model.eval()
# ok finally it lopads the model in cuda


`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

cuda:0


Qwen2VLForConditionalGeneration(
  (visual): Qwen2VisionTransformerPretrainedModel(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
    )
    (rotary_pos_emb): VisionRotaryEmbedding()
    (blocks): ModuleList(
      (0-31): 32 x Qwen2VLVisionBlock(
        (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): VisionSdpaAttention(
          (qkv): Linear8bitLt(in_features=1280, out_features=3840, bias=True)
          (proj): Linear8bitLt(in_features=1280, out_features=1280, bias=True)
        )
        (mlp): VisionMlp(
          (fc1): Linear8bitLt(in_features=1280, out_features=5120, bias=True)
          (act): QuickGELUActivation()
          (fc2): Linear8bitLt(in_features=5120, out_features=1280, bias=True)
        )
      )
    )
    (merger): PatchMerger(
      (ln_q): LayerNorm((1280,), eps=1e-06, elementwise_affin

In [ ]:
# dummy run as before
messages = [
{
    "role":"user",
    "content":[
        {
            "type":"image",
            "image":"https://images.chesscomfiles.com/uploads/v1/images_users/tiny_mce/Ognian_Mikov/php2nnXz9.png",

        },
        {
            "type":"text","text":"Provide surrounding boxes for all pawns in this image and respond in <|bbox|> format"
        },
    ]
}
]



text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
text



'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>Provide surrounding boxes for all pawns in this image and respond in <|bbox|> format<|im_end|>\n<|im_start|>assistant\n'

In [ ]:

image_inputs,video_inputs = process_vision_info(messages)
inputs = processor(text=[text],images=image_inputs,videos=video_inputs,padding=True,return_tensors="pt")
inputs = inputs.to(model.device)

In [ ]:
print("allocated:", torch.cuda.memory_allocated()/1024**3, "GB")
print("reserved:", torch.cuda.memory_reserved()/1024**3, "GB")
print(model.device)

allocated: 0.0 GB
reserved: 0.0 GB
cuda:0


In [ ]:
print(f"Total VRAM: {torch.cuda.get_device_properties(0).total_memory/1024**3:.2f} GB")

Total VRAM: 31.37 GB


In [ ]:

generated_ids = model.generate(**inputs, max_new_tokens=128)
# Step 1: Trimming - Remove the input prompt from output
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]

output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

['```json\n[\n  {"bbox": [12, 599, 72, 780]},\n  {"bbox": [12, 100, 72, 171]},\n  {"bbox": [12, 141, 72, 212]},\n  {"bbox": [12, 243, 72, 314]},\n  {"bbox": [12, 355, 72, 426]},\n  {"bbox": [12, 467, ']


In [ ]:
print("allocated:", torch.cuda.memory_allocated()/1024**3, "GB")
print("reserved:", torch.cuda.memory_reserved()/1024**3, "GB")

allocated: 0.0 GB
reserved: 0.0 GB


In [ ]:
print(model.device)

cuda:0


In [ ]:
!nvidia-smi

Sat Nov 22 00:44:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.06             Driver Version: 570.124.06     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5090        On  |   00000000:01:00.0 Off |                  N/A |
|  0%   31C    P8             23W /  575W |   10028MiB /  32607MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
